In [79]:
import pandas as pd
import numpy as np
import os
from env import host, username, password
import env
from pydataset import data

# exercises Pt.1

# Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [84]:
def get_db_url(host, username, password, db):
    
    '''
    return a formatted string containing username,
    password, host and db for MySQL SERVER AND DB.
    The env_file check to see if the env.py exist in current working directory
    
    '''
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'
    
    # create the url variable (string) with my username, password and host inserted
 

In [85]:
url = get_db_url(host, username, password, 'employees')

# Use your function to obtain a connection to the employees database.

In [87]:
query = 'SELECT * FROM employees LIMIT 10'

In [88]:
pd.read_sql(query,url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
5,10006,1953-04-20,Anneke,Preusig,F,1989-06-02
6,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
7,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
8,10009,1952-04-19,Sumant,Peac,F,1985-02-18
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


# Once you have successfully run a query:

# a. Intentionally make a typo in the database url. What kind of error message do you see?

In [ ]:
# error out

# b. Intentionally make an error in your SQL query. What does the error message look like?

In [ ]:
#error out

# Read the employees and titles tables into two separate DataFrames.

In [89]:
emp_query = 'SELECT * FROM employees LIMIT 100'

In [91]:
employees = pd.read_sql(emp_query,url)

In [93]:
type(employees)

pandas.core.frame.DataFrame

In [51]:
tit_query = 'SELECT * FROM titles'

In [94]:
titles = pd.read_sql(tit_query,url)

In [95]:
type(titles)

pandas.core.frame.DataFrame

# How many rows and columns do you have in each DataFrame? Is that what you expected?

In [96]:
employees.shape

(100, 6)

In [99]:
titles.shape

(443308, 4)

# Display the summary statistics for each DataFrame.


In [101]:
titles.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


In [100]:
employees.describe()

,emp_no
count,100.000000
mean,10050.500000
std,29.011492
min,10001.000000
25%,10025.750000
50%,10050.500000
75%,10075.250000
max,10100.000000


# How many unique titles are in the titles DataFrame?

In [102]:
titles['title'].unique()

array(['Senior Engineer', 'Staff', 'Engineer', 'Senior Staff',
       'Assistant Engineer', 'Technique Leader', 'Manager'], dtype=object)

# What is the oldest date in the to_date column?

In [200]:
titles.sort_values("to_date").agg('min')

emp_no                    10001
title        Assistant Engineer
from_date            1985-02-18
to_date              1987-01-27
dtype: object

# what is the most recent date in the to_date column?

In [103]:
titles.groupby("to_date").agg('min').tail(2)

,emp_no,title,from_date
to_date,,,
2002-08-01,20278,Assistant Engineer,1985-02-10
9999-01-01,10001,Assistant Engineer,1985-02-01


# exercises Pt.2

In [ ]:
# Copy the users and roles DataFrames from the examples above.

In [104]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [105]:
roles = pd.DataFrame({
    'id':[1,2,3,4],
    'name':['admin','author','reviewer','commenter']

})

roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


# What is the result of using a right join on the DataFrames?

In [106]:
users.merge(roles,
            left_on = "role_id",
            right_on ='id',
            how = 'right',
           indicator = True)


,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


# #What is the result of using a outer join on the DataFrames?

In [107]:
users.merge(roles,
            left_on = "role_id",
            right_on ='id',
            how = 'outer',
           indicator = True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


In [ ]:
#What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [108]:
(users.merge(roles,
            left_on = "role_id",
            right_on ='id',
            how = 'outer',
           indicator = True
           )
.drop(columns = "id_y")
.drop(columns = "role_id")
 
)

,id_x,name_x,name_y,_merge
0,1.0,bob,admin,both
1,2.0,joe,author,both
2,3.0,sally,reviewer,both
3,4.0,adam,reviewer,both
4,5.0,jane,NaN,left_only
5,6.0,mike,NaN,left_only
6,NaN,NaN,commenter,right_only


In [ ]:
#Load the mpg dataset from PyDataset.

In [6]:
mpg = data('mpg')

In [408]:
#Output and read the documentation for the mpg dataset.

In [7]:
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [8]:
#How many rows and columns are in the dataset?
mpg.shape

(234, 11)

In [13]:
#Check out your column names and perform any cleanup you may want on them.
mpg.rename(columns={'cty': 'city'},inplace = True)
mpg.rename(columns={'hwy': 'highway'},inplace = True)

In [14]:
#Display the summary statistics for the dataset.
mpg.describe()

,displ,year,cyl,city,highway
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


In [10]:
#How many different manufacturers are there?
len(mpg.groupby("manufacturer"))


15

In [9]:
mpg['manufacturer'].nunique()

15

In [84]:
mpg.head(1)

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact


In [95]:
#How many different models are there?
len(mpg.groupby("model"))


38

In [11]:
mpg['model'].nunique()

38

In [98]:
#Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car
mpg["mileage_difference"] = mpg['highway'] - mpg['city']
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


In [15]:
 #Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.
mpg["average_mileage"] = (mpg['highway'] + mpg['city']) /2
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,22.0


In [17]:
mpg['trans'].unique()

array(['auto(l5)', 'manual(m5)', 'manual(m6)', 'auto(av)', 'auto(s6)',
       'auto(l4)', 'auto(l3)', 'auto(l6)', 'auto(s5)', 'auto(s4)'],
      dtype=object)

In [16]:
#Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.
mpg["is_auto"] = (mpg["trans"].str.startswith("a"))
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,class,average_mileage,is_auto
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,22.0,False


In [23]:
#Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?
mpg.groupby("manufacturer").average_mileage.mean().sort_values(ascending = False).head(1)


manufacturer
honda    28.5
Name: average_mileage, dtype: float64

In [136]:
#Do automatic or manual cars have better miles per gallon?
mpg.groupby('is_auto').average_mileage.agg(['min','max','mean'])

,min,max,mean
is_auto,,,
False,10.5,39.5,22.227273
True,10.5,35.0,19.130573


# exercises Pt 3

 # Use your get_db_url function to help you explore the data from the chipotle database.

In [117]:
chip_url = get_db_url(host, username, password, 'chipotle')

In [119]:
chip_query = 'SELECT * FROM orders'

In [122]:
chip = pd.read_sql(chip_query,chip_url)

# What is the total price for each order?

In [123]:
chip['item_price'].str.contains(',').sum()

0

In [34]:
chip_df['item_price'] = chip_df['item_price'].str.replace('$', '')

/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_15218/3316443482.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  chip_df['item_price'] = chip_df['item_price'].str.replace('$', '')


In [124]:
chip[['item_price']] = chip_df[['item_price']].astype(float)

In [125]:
chip.groupby('order_id')['item_price'].agg('sum')

order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: item_price, Length: 1834, dtype: float64

# What are the most popular 3 items

In [44]:
chip.groupby('item_name')['id'].count().sort_values().tail(3)

item_name
Chips and Guacamole    479
Chicken Burrito        553
Chicken Bowl           726
Name: id, dtype: int64

# Which item has produced the most revenue?

In [126]:
chip.groupby('item_name')['item_price'].agg('sum').nlargest(1)


item_name
Chicken Bowl    7342.73
Name: item_price, dtype: float64

# Join the employees and titles DataFrames together.

In [127]:
titles.head(1)

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01


In [128]:
employees.head(1)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26


In [130]:
merg_df = employees.merge(titles,
             left_on = "emp_no",
             right_on ='emp_no',
             
             indicator = True)
             

In [299]:
merg_df

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date,employed_status,_merge
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,quit,both
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01,quit,both
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01,quit,both
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01,quit,both
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01,quit,both
5,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12,Senior Staff,1996-09-12,9999-01-01,quit,both
6,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12,Staff,1989-09-12,1996-09-12,quit,both
7,10006,1953-04-20,Anneke,Preusig,F,1989-06-02,Senior Engineer,1990-08-05,9999-01-01,quit,both
8,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10,Senior Staff,1996-02-11,9999-01-01,quit,both
9,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10,Staff,1989-02-10,1996-02-11,quit,both



# For each title, find the hire date of the employee that was hired most recently with that title.

In [131]:
merg_df['hire_date'] = pd.to_datetime(merg_df['hire_date'])
merg_df['birth_date'] = pd.to_datetime(merg_df['birth_date'])

merg_df['from_date'] = pd.to_datetime(merg_df['from_date'])



In [132]:
merg_df.groupby('title').hire_date.agg(max)

title
Assistant Engineer   1997-05-19
Engineer             1995-12-15
Senior Engineer      1995-12-15
Senior Staff         1995-03-13
Staff                1999-04-30
Technique Leader     1994-05-21
Name: hire_date, dtype: datetime64[ns]

# Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

In [134]:
cross_query = '''
            SELECT t.title, d.dept_name
            FROM dept_emp
            JOIN titles t USING(emp_no)
            JOIN departments d USING(dept_no)
            WHERE t.to_date > NOW();
            '''

In [135]:
cross_url = get_db_url(host, username, password, 'employees')

In [136]:
df_cross = pd.read_sql(cross_query, cross_url)

df_cross.head()

,title,dept_name
0,Senior Engineer,Development
1,Staff,Sales
2,Senior Engineer,Production
3,Senior Engineer,Production
4,Senior Staff,Human Resources


In [139]:
pd.crosstab(df_cross['title'],df_cross['dept_name'].T)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,68,1833,0,0,0,1542,433,77,0
Engineer,627,15677,0,0,0,13325,3744,830,0
Manager,1,1,1,1,1,1,1,1,1
Senior Engineer,1790,43364,0,0,0,37156,10390,2250,0
Senior Staff,12349,1085,10650,10843,12371,1123,0,10219,32032
Staff,3902,315,3199,3416,3880,349,0,3206,9967
Technique Leader,241,6117,0,0,0,5210,1422,321,0
